In [1]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
from pydub.silence import detect_nonsilent
from playsound import playsound
import py_string_tool as pst
from typing import Union,Dict,Tuple, List, Literal
import python_wizard as pw
import video_toolkit as vt


In [2]:
def text_to_milisecond(time_text:Union[str,int,float],delimiter:str = ".") -> Union[int,float]:
    """
    time_text should be seperated by dot for 
    convert strings to miliseconds to easily convert back and forth between video view and pydub input
    if it's already int it would return the same
    
    Convert time text to milliseconds.

    Args:
    time_text (Union[str, int, float]): Time in format "hr.min.sec" or "min.sec" or milliseconds.

    Returns:
    Union[int, float]: Time in milliseconds.

    Examples:
    "4.32" => (4*60 + 32) * 1000 = 272000 ms (4 min 32 sec)
    "1.40.32" => (1*3600 + 40*60 + 32) * 1000 = 6032000 ms (1 hr 40 min 32 sec)
    """
    if isinstance(time_text, (int, float)):
        return time_text

    if not isinstance(time_text, str):
        raise ValueError("Input must be a string, int, or float.")

    parts = time_text.split(delimiter)
    
    if len(parts) == 2:
        minutes, seconds = map(int, parts)
        return (minutes * 60 + seconds) * 1000
    elif len(parts) == 3:
        hours, minutes, seconds = map(int, parts)
        return (hours * 3600 + minutes * 60 + seconds) * 1000
    else:
        raise ValueError("Invalid time format. Use 'min.sec' or 'hr.min.sec'.")

def ms_to_time_text(milliseconds: Union[int, float]) -> str:
    """
    Convert milliseconds to time text format.

    Args:
    milliseconds (Union[int, float]): Time in milliseconds.

    Returns:
    str: Time in format "hr.min.sec" or "min.sec".

    Examples:
    272000 => "4.32" (4 min 32 sec)
    6032000 => "1.40.32" (1 hr 40 min 32 sec)
    """
    if not isinstance(milliseconds, (int, float)):
        raise ValueError("Input must be an integer or float representing milliseconds.")

    total_seconds = int(milliseconds / 1000)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}.{minutes:02d}.{seconds:02d}"
    else:
        return f"{minutes}.{seconds:02d}"
    

def export_audio(audio_segment:AudioSegment,
                 start_end_time_dict: Dict[int,Tuple[int,int]],
                 output_names:Dict[int,str],
                 output_folder:str = "",
                 progressbar:bool = True,
                 ) -> None:
    
    # medium tested
    """
    Key feature: 
        1) Remove the invalid path in output_names automatically
    the timestamp should be in miliseconds units(for now)

    
    export multiple audio_segments
    make sure that index in output_names is also in start_end_time_dict

    # example of start_end_time_dict
    start_end_time_dict = {
        6:  [14_633 , 15_933],
        7:  [24_455 , 25_534],
        8:  [25_700 , 27_550],
        9:  [27_899 , 30_000],
        10: [31_075 , 32_863],
        11: [33_439 , 36_188],
        12: [37_280 , 42_100],
        14: [42_865 , 47_224],
        
    
    TOADD: replace => it would check if file already exists, if so depending on it's True or False, it would replace the file
    """
    import py_string_tool as pst
    clean_output_names = {}
    for inx, output_name in output_names.items():
        clean_output_names[inx] = pst.clean_filename(output_name)
    
    from tqdm import tqdm
    if progressbar:
        loop_obj = tqdm(start_end_time_dict.items())
    else:
        loop_obj = start_end_time_dict.items()
    
    for inx, time_stamp in loop_obj:
        start_time, end_time = time_stamp
        try:
            output_name = clean_output_names[inx]
        except KeyError:
            raise KeyError(f"there's no index {inx} in your output_names(Dict). Please check your index again.")
        output_path = output_folder + "/" + output_name
        curr_audio = audio_segment[start_time:end_time]
        
        try:
            curr_audio.export(output_path)
        except PermissionError:
            raise KeyError(f"Please close the file {output_path} first.")
            
    
    

In [10]:
def test_export_audio():
    filepath = r"G:\My Drive\G_Videos\Learn French\Learn to speak French in 5 minutes - a dialogue for beginners!.mp3"
    audio = AudioSegment.from_file(filepath) 

    OUTPUT_FOLDER:str = "G:\My Drive\G_Videos\Learn French\Pydub Export test01"

    start_time = 35 * 1000  # Start at 35 seconds
    end_time = (1*60 + 35) * 1000    # End at 1 minute and 35 seconds
    
    manual_edit = {
        6:  [14_633 , 15_933],
        7:  [24_455 , 25_534],
        8:  [25_700 , 27_550],
        9:  [27_899 , 30_000],
        10: [31_075 , 32_863],
        11: [33_439 , 36_188],
        12: [37_280 , 42_100],
        14: [42_865 , 47_224],
        
        }
    
    # output_names01 has no index 10
    output_names01 = {
        6:  "01.01_I'm....mp3",
        7:  "01.02_Pleased to meet you.mp3",
        8:  "01.03_That's a nice name.mp3",
        9:  "01.05_Can I ask you a question?.mp3",
        11: "01.06_What do you like to do on a weekend?.mp3",
        12: "01.07_I like to learn French and read and you?.mp3",
        14: "01.08_I like to watch television.mp3",
        
        }
    
    output_names02 = {
        6:  "01.01_I'm....mp3",
        7:  "01.02_Pleased to meet you.mp3",
        8:  "01.03_That's a nice name.mp3",
        9:  "01.05_Can I ask you a question?.mp3",
        10: "01.07_Yes, of course.mp3",
        11: "01.08_What do you like to do on a weekend?.mp3",
        12: "01.09_I like to learn French and read and you?.mp3",
        14: "01.10_I like to watch television.mp3",
        
        }
    
    output_names03 = {
        6:  "01.01_I'm....wav",
        7:  "01.02_Pleased to meet you.wav",
        8:  "01.03_That's a nice name.wav",
        9:  "01.05_Can I ask you a question?.wav",
        10: "01.07_Yes, of course.mp3",
        11: "01.08_What do you like to do on a weekend?.wav",
        12: "01.09_I like to learn French and read and you?.wav",
        14: "01.10_I like to watch television.wav",
        
        }
    
    # Extract the segment from the audio
    segment = audio[start_time:end_time]
    try:
        export_audio(segment, manual_edit, output_names01,output_folder=OUTPUT_FOLDER)
    except Exception as error:
        assert isinstance(error, KeyError)
        
    export_audio(segment, manual_edit, output_names02,output_folder=OUTPUT_FOLDER)
    export_audio(segment, manual_edit, output_names03,output_folder=OUTPUT_FOLDER)

def test_text_to_milisecond():
    import inspect_py as inp
    actual01 = text_to_milisecond("4.32") # Output: 272000
    actual02 = text_to_milisecond("1.40.32")  # Output: 6032000
    actual03 = text_to_milisecond(5000)  # Output: 5000
    
    expect01 = 272_000
    expect02 = 6032000
    expect03 = 5000
    
    assert actual01 == expect01, inp.assert_message(actual01, expect01)
    assert actual02 == expect02, inp.assert_message(actual02, expect02)
    assert actual03 == expect03, inp.assert_message(actual03, expect03)

# test_text_to_milisecond()

# test_export_audio()

In [169]:
alarm_path = r"H:\D_Music\Sound Effect positive-logo-opener.mp3"
filepath = r"G:\My Drive\G_Videos\Learn French\French Phrases Video\02 French Phrases youll hear EVERYDAY in French conversations.mp4"
OUTPUT_FOLDER:str = r"G:\My Drive\G_Videos\Learn French\Pydub Export\02 Common Phrases"

In [ ]:
audio = AudioSegment.from_file(filepath) 

In [3]:
start_time = 35 * 1000  # Start at 35 seconds
end_time = (1*60 + 35) * 1000    # End at 1 minute and 35 seconds


In [4]:
time_in_ms:int = 28_000
vt.ms_to_time_text(time_in_ms)

'0.28'

In [279]:
time_in_str:str = "4.58"
vt.text_to_milisecond(time_in_str)
print(f"{vt.text_to_milisecond(time_in_str):_}")
print(f"{vt.text_to_milisecond(time_in_str)+2000:_}")

298_000
300_000


In [291]:
output_names = {
    1:  "01_What's up_Quoi de neuf.mp3",
    2:  "02_Long time no see_Ça fait longtemps.mp3",
    3:  "03_It works_It works/ok.mp3",
    4:  "04_I can't make it(I can't come)_Je peux pas venir.mp3",
    5.1:  "05.1_I'm on my way_Je suis en chemin.mp3",
    5.2:  "05.2_I'm on my way_Je suis en route.mp3",
    5.3:  "05.3_I'm on my way_J'arrive.mp3",
    6:  "06_I'm not far_Je suis pas loin.mp3",
    7:  "07_Good luck_Bonne chance.mp3",
    8:  "08_Get well soon_Bon rétablissement.mp3",
    9:  "09_Keep up the good work_Bravo, Continuez comme ça.mp3",
    10.1:  "10_It's crazy_C'est dingue.mp3",
    10.2:  "10_It's crazy_C'est fou.mp3",
    11:  "11_It's so cute. C'est trop mignon.mp3",
    12:  "12_I'll think about it_Je vais y réfléchir.mp3",
    13:  "13_I'm busy_Je suis occupé.mp3",
    14:  "14_It's too expensive_C'est trop cher.mp3",
    15:  "15_See you later_À plus.mp3",
    
    }


In [293]:
# cut 6
manual_edit = {
    1:  [28_000 , 29_000],
    2:  [43_500 , 45_000],
    3:  [59_000 , 60_000],
    4:  [76_000 , 77_000], 
    5.1:  [99_000 , 100_500],
    5.2:  [101_000 , 102_000],
    5.3:  [103_000 , 103_500],
    6: [121_500 , 122_500],
    7: [149_000 , 150_000],
    8: [159_000 , 160_400],
    9: [181_000 , 183_500],
    10.1: [202_000 , 203_500],
    10.2: [204_000 , 205_500],
    11: [219_800 , 221_000],
    12: [231_000 , 233_000],
    13: [260_000 , 261_000],
    14: [278_000 , 279_700],
    15: [297_500 , 298_500],
    
    }
manual_index = max(manual_edit.keys())

# manual_index = 10.2

In [294]:
# if your index is higher than it would crash your kernel
if manual_edit[manual_index][0] > manual_edit[manual_index][1]:
    raise Exception(f"check your index make sure first index is less than second index")

test_audio02 = audio[manual_edit[manual_index][0] : manual_edit[manual_index][1]]
play(test_audio02)


In [296]:
# play(chunks[audio_index])
vt.export_audio(audio, manual_edit, output_names,output_folder=OUTPUT_FOLDER)

# plot_loudness(segment)


100%|██████████| 18/18 [00:04<00:00,  4.15it/s]
